In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

To start this whole thing out, I need to collect the urls from wikipedia that have contestant data. I do this for both the Bachelor and Bachelorette.

In [2]:
import numpy as np
bachelor_seasons = np.arange(9,21)

bachelor_urls = []
for season in bachelor_seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelor_(season_{})'.format(season)
    bachelor_urls.append(season_url)

In [3]:
import numpy as np
bachelorette_seasons = np.arange(4,13)
bachelorette_seasons = np.insert(bachelorette_seasons,0,2)

bachelorette_urls = []
for season in bachelorette_seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelorette_(season_{})'.format(season)
    bachelorette_urls.append(season_url)

Let's create a function that takes in a url and then creates a soup from the html. This will come in handy when we are collecting the data.

In [4]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [58]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key of seasons
    tbl_d['season'] = [num] * len(tbl_d['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\w+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # find the first name with last name abbreviation
    name_abbreviation = []
    
    for name in tbl_d['name']:
        names = name.split(" ")

        new_names = []
        for name in names:
            cleaned_name = re.sub(r'\(\w+\)', ' ', name)
            new_names.append(cleaned_name)
        filtered = filter(lambda items: items.strip(), new_names) # remove blank space items 
        new_names = list(filtered)

        if len(new_names)== 1:
            new_name = new_names[0]
        else:
            new_name = "{} {}.".format(new_names[0], new_names[-1][0])
        name_abbreviation.append(new_name)
        
    tbl_d['name_abbreviation'] = name_abbreviation
    
    new_hometowns = []
    for hometown in tbl_d['hometown']:
        cleaned_hometown = re.sub(r'\[\w+\]', ' ', hometown)
        new_hometowns.append(cleaned_hometown)

    tbl_d['hometown'] = new_hometowns
    
    # return dictionary
    return tbl_d

In [59]:
def frameMaker(urls):
    frames = []

    for url in urls:
        dictionary = dataCollector(url)
        frame = pd.DataFrame(dictionary)
        frames.append(frame)

    combined = pd.concat(frames)
    
    return combined.reset_index()

In [60]:
bachelorette_frame = frameMaker(bachelorette_urls)

In [61]:
bachelor_frame = frameMaker(bachelor_urls)

In [62]:
bachelor_frame.tail(50)

,index,age,elimination,hometown,name,name_abbreviation,occupation,season
268,8,26,Eliminated in week 6,"Great Falls, Virginia",Ashley Iaconetti,Ashley I.,Nanny/Freelance Journalist,19
269,9,21,Eliminated in week 5,"Maple Valley, Washington",Mackenzie Deonigi,Mackenzie D.,Dental Assistant,19
270,10,27,Eliminated in week 5,"Indianapolis, Indiana",Samantha Steffen,Samantha S.,Fashion Designer,19
271,11,24,Eliminated in week 4,"Vinings, Georgia",Ashley Salter,Ashley S.,Hair Stylist,19
272,12,30,Eliminated in week 4,"Beaverton, Oregon",Juelia Kinney,Juelia K.,Esthetician,19
273,13,26,Eliminated in week 4,"Hasbrouck Heights, New Jersey",Nikki Delventhal,Nikki D.,Former NFL Cheerleader,19
274,14,25,Eliminated in week 4,"Howland, Ohio",Jillian Anderson,Jillian A.,News Producer,19
275,15,29,Eliminated in week 3,"Kankakee, Illinois",Amber James,Amber J.,Bartender,19
276,16,29,Eliminated in week 3,"Wellington, Florida",Tracy Darakis,Tracy D.,Fourth Grade Teacher,19
277,17,33,Eliminated in week 3,"Algonquin, Illinois",Trina Scherenberg,Trina S.,Special Education Teacher,19


In [43]:
bachelorette_frame.to_csv('data/bachelorette_contestants.csv', index = False)

In [44]:
bachelor_frame.to_csv('data/bachelor_contestants.csv', index = False)